In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import gender_guesser.detector as gender

#### Raspagem: lista de mulheres nomeadas e ganhadoras do Oscar em categorias em que gêneros competem.

**Obs.: Essa foi uma raspagem inicial de uma página da Wikipedia. No projeto final, a fonte usada foi a base de dados do Oscar mostrada mais abaixo.**

Como alguns elementos ocupam mais que uma linha, não é possível fazer a raspagem iterando entre os múltiplos de cada célula. A solução encontrada foi:

- Para o ano ('year'): tentar converter os 4 primeiros caracteres da célula em inteiro. Se não houver erro, é porque a célula é um ano válido; ele é salvo na variável `new_year`.
- Para o status ('Nominated' ou 'Won'): todas as células de status possuem `<td class="no table-no2" (...)>` para _Nominated_ ou `<td class="yes table-yes2" (...)>` para _Won_. Verifica-se se a célula atual é uma dessas e, se sim, o texto dela, da categoria (título da tabela) e do ano `new_year` é adicionado em suas respectivas listas.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_female_Academy_Award_winners_and_nominees_for_non-gendered_categories"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

year = []
category = []
status = []
all_tables = soup.find_all(class_='wikitable')

for table in all_tables:
	td = table.find_all('td')
	new_year = 0
	for i in range(0, len(td)):
		try:
			old_year = new_year
			new_year = int(td[i].text[0:4])
			if new_year < 1000:
				new_year = old_year
		except ValueError:
			pass
		if td[i].get('class'):
			status.append(td[i].text[0:-1])
			category.append(td[0].text[18:-1])
			if new_year != 0:
				year.append(new_year)

oscar_women_df = pd.DataFrame({
	'year': year,
	'category': category,
	'status': status
})

#	renomeia as categorias que possuíam sobrescrito no título.
oscar_women_df.category = oscar_women_df.category.replace({
	'Best Sound [note 12]': 'Best Sound Mixing',
	'Best Original Score [note 7]': 'Best Original Score'
})

#	corrige um typo na tabela original.
oscar_women_df.status = oscar_women_df.status.replace({'Nominated}': 'Nominated'})

oscar_women_df


,year,category,status
0,2007,Best Animated Feature,Nominated
1,2011,Best Animated Feature,Nominated
2,2012,Best Animated Feature,Won
3,2013,Best Animated Feature,Won
4,2013,Best Animated Feature,Nominated
...,...,...,...
1418,2017,Best Original Screenplay,Nominated
1419,2017,Best Original Screenplay,Nominated
1420,2018,Best Original Screenplay,Nominated
1421,1917,Best Original Screenplay,Nominated


# Raspagem: [base de dados da Academia de Artes e Ciências Cinematográficas](https://awardsdatabase.oscars.org/).
Devido à dinamicidade do site, para a automação completa seria necessária o uso do Selenium. Para os fins do projeto, o código da página de busca avançada foi salvo nos arquivos `nominees-men.html` e `nominees-women.html`. As páginas possuem as seguintes categorias:

**Páginas salvas:**
1. Best Picture
2. Directing
3. Writing
4. Cinematography
5. Costume Design
6. Produdction Design
7. Documentary Feature
8. Film Editing
9. Makeup
10. International Feature Film
11. Visual Effects

### A FAZER:
[ ] Quando um diretor é indicado a dois filmes, somente o primeiro filme é minerado (e terminado por ';')

In [151]:
# Cria o dataframe para páginas de buscas com múltiplas categorias.
def get_oscar_df(file):
	f = open(file)
	soup = BeautifulSoup(f, 'html.parser')
	df = pd.DataFrame({'year': [], 'category': [], 'film': [], 'name': [], 'status': []})

	all_groups = soup.find_all(class_='awards-result-chron result-group group-awardcategory-chron')
	for group in all_groups:
		year = group.find(class_="result-group-title").text.strip()[0:4]
		all_categories = group.find_all(class_="result-subgroup subgroup-awardcategory-chron")
		for category in all_categories:
			all_films = category.find_all(class_="result-details awards-result-actingorsimilar")
			category = category.find(class_="result-subgroup-title").text.strip()
			for row in all_films:
				try:
					film = row.find(class_="awards-result-film-title").text.strip()
				except AttributeError:
					film = 'Error'
				if row.find(class_="glyphicon glyphicon-star"):
					status = 'winner'
				else:
					status = 'nominated'
				all_names = re.split(r'; |: | and | & | by |, ', row.find(class_="awards-result-nominationstatement").text)
				for name in all_names:
					name = name.strip(' \n()')
					row = pd.DataFrame({'year': [year],
										'category': [category],
										'film': [film],
										'name': [name],
										'status': [status]
										})
					df = pd.concat([df, row], ignore_index=True)

	to_remove = ['Jr.', 'Producer', 'Producers', 'Adaptation', 'Art Direction',
				 'Interior Decoration', 'Set Decoration', 'Production Design',
				 'Screenplay', 'Written', 'Story', 'Stories',
				 'Adapted for the screen', 'Written for the screen']
	df = df[-df.name.isin(to_remove)]
	f.close()
	return df

In [156]:
# salva para csv:
# women_df = get_oscar_df('oscar-pages/nominees-women.html')
# men_df = get_oscar_df('oscar-pages/nominees-men.html')
# women_df.to_csv('oscar-women-v0.csv', index=False)
# men_df.to_csv('oscar-men-v0.csv', index=False)


#### limpando os datasets
Para categorias em que homens e mulheres eram indicados pelo mesmo filme, os indicados apareciam tanto na busca por homens quanto por mulheres. escolheu-se então usar o pacote [`gender-guesser`](https://pypi.org/project/gender-guesser/) para limpar os nomes masculinos da tabela feminina e vice-versa.

Por esse método, 361 mulheres foram removidas do dataset masculino, e 487 homens do dataset feminino. Alguns nomes foram indicados mais de uma vez, então a alteração no tamanho dos datasets foi maior.

In [159]:
women_df = pd.read_csv('oscar-women-v0.csv')
men_df = pd.read_csv('oscar-men-v0.csv')

g = gender.Detector()

female_names = []
male_names = []

for name in women_df.name.unique():
	first_name = name.split()[0]
	if g.get_gender(first_name) == 'male' or g.get_gender(first_name) == 'mostly_male':
		male_names.append(name)
women_df = women_df[-women_df.name.isin(male_names)]

for name in men_df.name.unique():
	first_name = name.split()[0]
	if g.get_gender(first_name) == 'female' or g.get_gender(first_name) == 'mostly_female':
		female_names.append(name)
men_df = men_df[-men_df.name.isin(female_names)]

# converte anos para int
women_df.year.astype(int)
men_df.year.astype(int)

print("Qntd. de nomes femininos removidos: %d" % len(female_names))
print("Qntd. de nomes masculinos removidos: %d" % len(male_names))


Qntd. de nomes femininos removidos: 361
Qntd. de nomes masculinos removidos: 487


In [158]:
# salva em um csv final
# women_df.to_csv('oscar-women-v1.csv', index=False)
# men_df.to_csv('oscar-men-v1.csv', index=False)

#### A FAZER:
- [ ] colocar colunas `male` e `female` nos respectivos datasets e uni-los em um único (checar por células idênticas).
- [ ] começar gráficos!